This code takes a plate boundary file (in this case subduction zones) and extrats the latitude, longitude co-ordinates of those polylines (for each segment) at each time and then stores them in a dictionary.  The lat-lon co-ords are a extracted from a tessellation of each segment.

It should be quite easy to change models and adapt for other 

Referneces 

Domeier, M. and Torsvik, T.H., 2014. Plate tectonics in the late Paleozoic. Geoscience Frontiers, 5(3), pp.303-350.

In [1]:
import pygplates
import numpy as np

In [8]:
#setbasedir for loading reconstruction files
basedir = '/Users/Andrew/Documents/PhD/Scripts/Python_Scripts/pyGPlates_examples/General_plate_reconstruction/Sample_data/'

#Domeier and Torsvik (2014)
DT14_rotation_model = pygplates.RotationModel('%sDT14_rotations.rot' % basedir)
DT14_topologies = ['%sDT14_ridge.gpml' % basedir,
                   '%sDT14_subduction.gpml' % basedir,
                   '%sDT14_topologies.gpml' % basedir,
                   '%sDT14_transform.gpml' % basedir]

In [13]:
subduction_zone_geometries = dict()
times = np.arange(250, 420, 10)
for time in times:

    #print tmp_csv
    subduction_zone_geometries[time] = []

    # Resolve our topological plate polygons (and deforming networks) to the current 'time'.
    # We generate both the resolved topology boundaries and the boundary sections between them.

    #these arrays are to store the topologies and boundary sections
    resolved_topologies = []
    shared_boundary_sections = []

    pygplates.resolve_topologies(DT14_topologies, DT14_rotation_model, resolved_topologies, \
                                 time, shared_boundary_sections)

    # Iterate over the shared boundary sections (since one plate boundary can be shared by two plates)

    for  shared_boundary_section in shared_boundary_sections:

        #we only want subudction zones (NB you can change the feature type to transform/MoR etc.)
        if shared_boundary_section.get_feature().get_feature_type() == pygplates.FeatureType.gpml_subduction_zone:

            #iterate over sub segments in each feature
            for shared_sub_segment in shared_boundary_section.get_shared_sub_segments():
                shared_sub_segment_geometry = shared_sub_segment.get_resolved_geometry()
                shared_sub_segment_geometry_tessellated = shared_sub_segment_geometry.to_tessellated(np.radians(1))
                #convert to lat lon array, you can conver to a list or to a feature as well, depending
                #what output you want one may be better than others
                tmp = shared_sub_segment_geometry_tessellated.to_lat_lon_array()

            #append everything to a big dictionary
            subduction_zone_geometries[time].append(tmp)